1000 Steps Random Walk
---
Consider a 1000-state version of the random walk task. The states are numbered from `1 to 1000`, left to right, and all episodes begin near the center, in state `500`. State transitions are from the current state to one of the `100` neighboring states to its left, or to one of the `100` neighboring states to its right, all with equal probability. Of course, if the current state is near an edge, then there may be fewer than `100` neighbors on that side of it. In this case, all the probability that would have gone into those missing neighbors goes into the probability of terminating on that side (thus, state 1 has a 0.5 chance of terminating on the left, and state 950 has a 0.25 chance of terminating on the right). As usual, termination on the left produces a reward of `-1`, and termination on the right produces a reward of `+1`. All other transitions have a reward of `zero`.

<img style="float:left" src="semi-TD.png" alt="drawing" width="700"/>

For the state aggregation, the 1000 states were partitioned into 10 groups of 100 states each (i.e., states 1–100 were one group, states 101–200 were another, and so on)

<img style="float:left" src="update.png" alt="drawing" width="500"/>

In [2]:
import numpy as np

In [1]:
NUM_STATES = 1000
START = 500
END_0 = 0
END_1 = 1001

In [119]:
class LinearValueFunction:
    
    def __init__(self, order, method="poly"):
        if method == "poly":
            self.func = [lambda x, i=i: np.power(x, i) for i in range(0, order+1)]  # s^i
        if method == "fourier":
            self.func = [lambda x, i=i: np.cos(np.pi*x*i) for i in range(0, order+1)]  # cos(pi*s*i)
        self.weights = np.zeros(order+1)
    
    def value(self, state):
        state = state/NUM_STATES
        features = np.array([f(state) for f in self.func])
        return np.dot(features, self.weights)
    
    def update(self, delta, state):
        state = state/NUM_STATES
        dev = np.array([f(state) for f in self.func])
        self.weights += delta*dev

In [52]:
class AggValueFunction:
    
    def __init__(self, num_groups=10):
        self.num_groups = num_groups
        self.group_size = NUM_STATES // self.num_groups
        self.values = np.zeros(self.num_groups)
        
    def value(self, state):
        # explicitly set end state value
        if state == END_0:
            return -1
        if state == END_1:
            return 1
        group = (state - 1) // self.group_size
        value = self.values[group]
        return value
    
    def update(self, delta, state):
        dev = 1  # derivative is 1 in this case
        group = (state - 1) // self.group_size
        self.values[group] += delta*dev

In [73]:
class RandomWalk:
    
    def __init__(self, step=1, lr=2e-5, gamma=1, debug=True):
        self.state = START
        self.actions = ["left", "right"]
        self.end = False
        self.n = step
        self.lr = lr
        self.gamma = gamma
        self.debug = debug
        
    def chooseAction(self):
        action = np.random.choice(self.actions)
        return action
    
    def takeAction(self, action):
        # choose steps from 1 to 100
        steps = np.random.choice(range(1, 101))
        if action == "left":
            state = self.state - steps
        else:
            state = self.state + steps
        # judge if end of game
        if state <= END_0 or state >= END_1:
            self.end = True
            if state <= END_0:
                state = END_0
            else:
                state = END_1
        
        self.state = state
        return state
    
    def giveReward(self):
        if self.state == END_0:
            return -1
        if self.state == END_1:
            return 1
        return 0
    
    def reset(self):
        self.state = START
        self.end = False
    
    def play(self, valueFunction, rounds=1e5):
        for rnd in range(rounds):
            self.reset()
            t = 0
            T = np.inf
            action = self.chooseAction()
            
            actions = [action]
            states = [self.state]
            rewards = [0]
            while True:
                if t < T:
                    state = self.takeAction(action)  # next state
                    reward = self.giveReward()  # next state-reward
                    
                    states.append(state)
                    rewards.append(reward)
                    
                    if self.end:
                        if self.debug:
                            if (rnd+1) % 5000 == 0:
                                print("Round {}: End at state {} | number of states {}".format(rnd+1, state, len(states)))
                        T = t+1
                    else:
                        action = self.chooseAction()
                        actions.append(action)  # next action
                # state tau being updated
                tau = t - self.n + 1
                if tau >= 0:
                    G = 0
                    for i in range(tau+1, min(tau+self.n+1, T+1)):
                        G += np.power(self.gamma, i-tau-1)*rewards[i]
                    if tau+self.n < T:
                        state = states[tau+self.n]
                        G += np.power(self.gamma, self.n)*valueFunction.value(state)
                    # update value function
                    state = states[tau] 
                    delta = self.lr*(G - valueFunction.value(state))
                    valueFunction.update(delta, state)
                    
                if tau == T-1:
                    break
                
                t += 1

### Aggregate State

In [74]:
rw = RandomWalk(step=1, lr=0.001)
vFunc = AggValueFunction(num_groups=10)

In [75]:
rw.play(rounds=10000, valueFunction=vFunc)

Round 5000: End at state 0 | number of states 319
Round 10000: End at state 0 | number of states 74


In [76]:
vFunc.values

array([-0.70370298, -0.49182061, -0.32981767, -0.16929717, -0.02333602,
        0.09878731,  0.21842415,  0.35668983,  0.51019321,  0.69859723])

### General Linear Function

#### Poly

In [120]:
rw = RandomWalk(step=1, lr=0.1)
polyFunc = LinearValueFunction(order=5, method="poly")

In [121]:
rw.play(rounds=5000, valueFunction=polyFunc)

Round 5000: End at state 0 | number of states 31


In [122]:
states = []
for i in range(1, 1001):
    states.append(polyFunc.value(i))

#### Fourier

In [125]:
rw = RandomWalk(step=1, lr=0.001)
fFunc = LinearValueFunction(order=5, method="fourier")

In [126]:
rw.play(rounds=5000, valueFunction=fFunc)

Round 5000: End at state 1001 | number of states 362


In [127]:
states = []
for i in range(1, 1001):
    states.append(fFunc.value(i))